In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_FXOption.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CommCCY = 'USD'
TermCCY = 'TWD'
Type = 'OPT'
MTMDate = MktSheet.cell(1, 3).value
FXSpot = MktSheet.cell(3, 6).value

In [4]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
CommDC = ql.Actual360()
TermDC = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [5]:
CommZeroSP = MktSheet.cell(22, 6).value
CommZero3M = MktSheet.cell(23, 6).value
CommZero6M = MktSheet.cell(24, 6).value
CommZero1Y = MktSheet.cell(25, 6).value
print(CommZeroSP)

0.001459727696536763


In [6]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [7]:
TermZeroSP = MktSheet.cell(22, 12).value
TermZero3M = MktSheet.cell(23, 12).value
TermZero6M = MktSheet.cell(24, 12).value
TermZero1Y = MktSheet.cell(25, 12).value
print(TermZeroSP)

-0.00392904297456824


In [8]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y])
ListCommRate = list([CommZeroSP, CommZero3M, CommZero6M, CommZero1Y])
ListTermRate = list([TermZeroSP, TermZero3M, TermZero6M, TermZero1Y])
ListDate
ListCommRate

[0.001459727696536763,
 0.001459727696619909,
 0.002038942990682572,
 0.002656419500050458]

In [9]:
ListTermRate

[-0.00392904297456824,
 -0.003929042973135284,
 -0.0006520395000015291,
 0.001329938573603508]

In [10]:
VecDate = ql.DateVector(ListDate)
VecCommRate = ql.DoubleVector(ListCommRate)
VecTermRate = ql.DoubleVector(ListTermRate)

In [11]:
TSCommZero = ql.ZeroCurve(VecDate, VecCommRate, CommDC, Cal, ql.Linear(), ql.Continuous, ql.Annual)
h_TSCommZero = ql.YieldTermStructureHandle(TSCommZero)
TSTermZero = ql.ZeroCurve(VecDate, VecTermRate, TermDC, Cal, ql.Linear(), ql.Continuous, ql.Annual)
h_TSTermZero = ql.YieldTermStructureHandle(TSTermZero)

In [12]:
ListVolDate = list([Date6M, Date1Y])
ListStrike = list()
for i in range(9):
    ListStrike.append(MktSheet.cell(31+i, 15).value)
ListStrike

[26.75, 27, 27.25, 27.5, 27.75, 28, 28.25, 28.5, 28.75]

In [13]:
VecVolDate = ql.DateVector(ListVolDate)
VecStrike = ql.DoubleVector(ListStrike)
M_Vol = ql.Matrix(9, 2, 0.0)

In [14]:
for i in range(9):
    for j in range(2):
        M_Vol[i][j] = MktSheet.cell(31+i, 16+j).value
print(M_Vol)

| 0.215209 0.113231 |
| 0.138996 0.0627988 |
| 0.0627825 0.0579179 |
| 0.0535264 0.0570613 |
| 0.0524342 0.0562047 |
| 0.0527047 0.0561806 |
| 0.0542796 0.0573503 |
| 0.0558546 0.0585201 |
| 0.0868235 0.0596899 |



In [15]:
DC365 = ql.Actual365Fixed()
VolSurface = ql.BlackVarianceSurface(evDate, Cal, VecVolDate, VecStrike, M_Vol, DC365)
h_VolSurface = ql.BlackVolTermStructureHandle(VolSurface)

In [16]:
aVol = h_VolSurface.blackVol(0.75, 27.891)
print(aVol)

0.05500941759027123


In [17]:
OPTSheet = PstBook['Option']

In [18]:
start_row = 2
end_row = 4
start_col = 0
end_col = 11
recordcount = end_col - start_col + 1
ListOPT = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in OPTSheet[i][start_col:end_col+1]]
    ListOPT.append(row)
    print(row)

['OPT001', datetime.datetime(2021, 5, 19, 0, 0), datetime.datetime(2021, 5, 21, 0, 0), datetime.datetime(2021, 9, 21, 0, 0), 'Long', 'Call', 'VLN', 'USDTWD', 'USD', 'TWD', 28, 1000000]
['OPT002', datetime.datetime(2021, 6, 13, 0, 0), datetime.datetime(2021, 6, 15, 0, 0), datetime.datetime(2021, 12, 15, 0, 0), 'Short', 'Call', 'VLN', 'USDTWD', 'USD', 'TWD', 27.8, 1000000]
['OPT003', datetime.datetime(2021, 6, 21, 0, 0), datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), 'Long', 'Put', 'VLN', 'USDTWD', 'USD', 'TWD', 28.1, 1000000]


In [19]:
OPTBook = list()
for i in range(len(ListOPT)):
    row = ListOPT[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    record.append(row[8])
    record.append(row[9])
    record.append(row[10])
    record.append(row[11])
    OPTBook.append(record)

In [20]:
OPTBook

[['OPT001',
  Date(19,5,2021),
  Date(21,5,2021),
  Date(21,9,2021),
  'Long',
  'Call',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  28,
  1000000],
 ['OPT002',
  Date(13,6,2021),
  Date(15,6,2021),
  Date(15,12,2021),
  'Short',
  'Call',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  27.8,
  1000000],
 ['OPT003',
  Date(21,6,2021),
  Date(23,6,2021),
  Date(23,5,2022),
  'Long',
  'Put',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  28.1,
  1000000]]

In [21]:
for i in range(len(OPTBook)):
    #i = 0
    record = OPTBook[i]
    tradeDate = record[1]
    maturityDate = record[3]
    position = record[4]
    OptClass = record[5]
    OptType = record[6]
    VolClass = record[7]
    CommCCY = record[8]
    TermCCY = record[9]
    Strike = record[10]
    CommAmt = record[11]    
    EuExer = ql.EuropeanExercise(maturityDate)
    if (OptClass == 'Call'):
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Call, Strike)
    else:
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Put, Strike)    
    anEuroOption = ql.EuropeanOption(VLPayoff, EuExer)
    
    Q_Spot = ql.SimpleQuote(FXSpot)
    hQ_Spot = ql.QuoteHandle(Q_Spot)
    GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, 
                                                   h_TSTermZero, h_VolSurface)
    AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
    anEuroOption.setPricingEngine(AEEngine)
    if (position == 'Long'):
        Sign = 1.0
    else:
        Sign = -1.0
    
    Value = Sign * CommAmt * anEuroOption.NPV()
    print(Value)

214380.82208287125
-421230.21349021135
727194.1754706659
